In [1]:
import pandas as pd
import os
from utils import *
from utils import df_from_filings

In [2]:
from textblob import TextBlob

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

In [16]:
df = df_from_filings()
df.shape

(2644, 5)

In [17]:
%%time
vader_untrained_polarity = df.apply(lambda row: vader.polarity_scores(row['text']), axis=1)

CPU times: user 29min 7s, sys: 5.82 s, total: 29min 13s
Wall time: 29min 19s


In [18]:
%%time
vader_untrained_preprocessed_polarity = df.apply(lambda row: vader.polarity_scores(preprocess(row['text'])), axis=1)


CPU times: user 9min 1s, sys: 1.77 s, total: 9min 3s
Wall time: 9min 4s


In [42]:
vader_untrained_polarity = pd.DataFrame(vader_untrained_polarity.values.tolist())
vader_untrained_polarity = vader_untrained_polarity.add_prefix('vader_untrained_polarity_');

vader_untrained_preprocessed_polarity = pd.DataFrame(vader_untrained_preprocessed_polarity.values.tolist())
vader_untrained_preprocessed_polarity = vader_untrained_preprocessed_polarity.add_prefix(
    'vader_untrained_preprocessed_polarity_');

In [43]:
df = pd.concat([df, vader_untrained_polarity, vader_untrained_preprocessed_polarity], axis=1);

In [44]:
df.describe()

,vader_untrained_polarity_neg,vader_untrained_polarity_neu,vader_untrained_polarity_pos,vader_untrained_polarity_compound,vader_untrained_preprocessed_polarity_neg,vader_untrained_preprocessed_polarity_neu,vader_untrained_preprocessed_polarity_pos,vader_untrained_preprocessed_polarity_compound,vader_untrained_polarity_0,vader_untrained_polarity_1,...,textblob_preprocessed_sentiment_polarity,textblob_preprocessed_sentiment_subjectivity,vader_untrained_polarity_0,vader_untrained_polarity_1,vader_untrained_polarity_2,vader_untrained_polarity_3,vader_untrained_preprocessed_polarity_0,vader_untrained_preprocessed_polarity_1,vader_untrained_preprocessed_polarity_2,vader_untrained_preprocessed_polarity_3
count,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,...,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000
mean,0.035739,0.813275,0.150975,0.999997,0.052262,0.765345,0.182397,0.999982,0.035739,0.813275,...,0.067744,0.373874,0.035739,0.813275,0.150975,0.999997,0.052262,0.765345,0.182397,0.999982
std,0.010650,0.025489,0.021155,0.000018,0.014430,0.025553,0.023767,0.000045,0.010650,0.025489,...,0.019210,0.031392,0.010650,0.025489,0.021155,0.000018,0.014430,0.025553,0.023767,0.000045
min,0.011000,0.705000,0.065000,0.999700,0.017000,0.664000,0.071000,0.999400,0.011000,0.705000,...,-0.012325,0.167288,0.011000,0.705000,0.065000,0.999700,0.017000,0.664000,0.071000,0.999400
25%,0.028000,0.798000,0.138000,1.000000,0.041000,0.750750,0.167000,1.000000,0.028000,0.798000,...,0.054546,0.353527,0.028000,0.798000,0.138000,1.000000,0.041000,0.750750,0.167000,1.000000
50%,0.035000,0.814000,0.150000,1.000000,0.052000,0.765000,0.182000,1.000000,0.035000,0.814000,...,0.066431,0.374090,0.035000,0.814000,0.150000,1.000000,0.052000,0.765000,0.182000,1.000000
75%,0.042000,0.829000,0.163000,1.000000,0.062000,0.780000,0.197000,1.000000,0.042000,0.829000,...,0.080336,0.395777,0.042000,0.829000,0.163000,1.000000,0.062000,0.780000,0.197000,1.000000
max,0.082000,0.924000,0.240000,1.000000,0.129000,0.901000,0.268000,1.000000,0.082000,0.924000,...,0.146607,0.470028,0.082000,0.924000,0.240000,1.000000,0.129000,0.901000,0.268000,1.000000


In [30]:
#TODO lemma, stem, check if remove frequent is good

In [35]:
%%time
textblob_sentiment_polarity = df.apply(lambda row: TextBlob(row['text']).sentiment.polarity, axis=1)
textblob_sentiment_subjectivity = df.apply(lambda row: TextBlob(row['text']).sentiment.subjectivity, axis=1)

textblob_sentiment = pd.concat([textblob_sentiment_polarity, textblob_sentiment_subjectivity], axis=1)
textblob_sentiment.columns = ['textblob_sentiment_polarity', 'textblob_sentiment_subjectivity']

df = pd.concat([df, textblob_sentiment], axis=1);

CPU times: user 5min 40s, sys: 1.75 s, total: 5min 42s
Wall time: 5min 42s


In [36]:
%%time
textblob_preprocessed_sentiment_polarity = df.apply(lambda row: TextBlob(preprocess(row['text'])).sentiment.polarity,
                                                    axis=1)
textblob_preprocessed_sentiment_subjectivity = df.apply(
    lambda row: TextBlob(preprocess(row['text'])).sentiment.subjectivity, axis=1)

textblob_preprocessed_sentiment = pd.concat(
    [textblob_preprocessed_sentiment_polarity, textblob_preprocessed_sentiment_subjectivity], axis=1)
textblob_preprocessed_sentiment.columns = ['textblob_preprocessed_sentiment_polarity',
                                           'textblob_preprocessed_sentiment_subjectivity']

df = pd.concat([df, textblob_preprocessed_sentiment], axis=1);

CPU times: user 3min 19s, sys: 329 ms, total: 3min 20s
Wall time: 3min 20s


In [41]:
df.describe()

,vader_untrained_polarity_neg,vader_untrained_polarity_neu,vader_untrained_polarity_pos,vader_untrained_polarity_compound,vader_untrained_preprocessed_polarity_neg,vader_untrained_preprocessed_polarity_neu,vader_untrained_preprocessed_polarity_pos,vader_untrained_preprocessed_polarity_compound,vader_untrained_polarity_0,vader_untrained_polarity_1,...,vader_untrained_polarity_2,vader_untrained_polarity_3,vader_untrained_preprocessed_polarity_0,vader_untrained_preprocessed_polarity_1,vader_untrained_preprocessed_polarity_2,vader_untrained_preprocessed_polarity_3,textblob_sentiment_polarity,textblob_sentiment_subjectivity,textblob_preprocessed_sentiment_polarity,textblob_preprocessed_sentiment_subjectivity
count,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,...,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000,2644.000000
mean,0.035739,0.813275,0.150975,0.999997,0.052262,0.765345,0.182397,0.999982,0.035739,0.813275,...,0.150975,0.999997,0.052262,0.765345,0.182397,0.999982,0.064241,0.362712,0.067744,0.373874
std,0.010650,0.025489,0.021155,0.000018,0.014430,0.025553,0.023767,0.000045,0.010650,0.025489,...,0.021155,0.000018,0.014430,0.025553,0.023767,0.000045,0.017329,0.026845,0.019210,0.031392
min,0.011000,0.705000,0.065000,0.999700,0.017000,0.664000,0.071000,0.999400,0.011000,0.705000,...,0.065000,0.999700,0.017000,0.664000,0.071000,0.999400,0.005231,0.254626,-0.012325,0.167288
25%,0.028000,0.798000,0.138000,1.000000,0.041000,0.750750,0.167000,1.000000,0.028000,0.798000,...,0.138000,1.000000,0.041000,0.750750,0.167000,1.000000,0.051778,0.344731,0.054546,0.353527
50%,0.035000,0.814000,0.150000,1.000000,0.052000,0.765000,0.182000,1.000000,0.035000,0.814000,...,0.150000,1.000000,0.052000,0.765000,0.182000,1.000000,0.062954,0.364335,0.066431,0.374090
75%,0.042000,0.829000,0.163000,1.000000,0.062000,0.780000,0.197000,1.000000,0.042000,0.829000,...,0.163000,1.000000,0.062000,0.780000,0.197000,1.000000,0.075210,0.382567,0.080336,0.395777
max,0.082000,0.924000,0.240000,1.000000,0.129000,0.901000,0.268000,1.000000,0.082000,0.924000,...,0.240000,1.000000,0.129000,0.901000,0.268000,1.000000,0.133639,0.444787,0.146607,0.470028


In [46]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2022-05-31 19:26:19,347 loading file /Users/etiennebruno/.flair/models/sentiment-en-mix-distillbert_4.pt


In [56]:
%%time
def get_flair_sentiment(text):
    s = flair.data.Sentence(text)
    flair_sentiment.predict(s)
    return s.tag, s.score

df_flair_sentiment = df.apply(lambda row: get_flair_sentiment(row['text'])  , axis=1, result_type='expand')
df_flair_sentiment.columns = ['flair_sentiment_tag', 'flair_sentiment_score']
df = pd.concat([df, df_flair_sentiment], axis=1);

# Save data
df_flair_sentiment.to_pickle("flair_backup_2644.pkl")

CPU times: user 1h 16min 15s, sys: 34.2 s, total: 1h 16min 50s
Wall time: 1h 11min 42s


In [58]:
%%time
df_flair_sentiment_preprocessed = df.apply(lambda row: get_flair_sentiment(preprocess(row['text'])), axis=1, result_type='expand')
df_flair_sentiment_preprocessed.columns = ['flair_preprocessed_sentiment_tag', 'flair_preprocessed_sentiment_score']
df = pd.concat([df, df_flair_sentiment_preprocessed], axis=1);

# Save data
df_flair_sentiment_preprocessed.to_pickle("flair_preprocessed_backup_2644.pkl")

CPU times: user 1h 5min 49s, sys: 24.9 s, total: 1h 6min 14s
Wall time: 1h 1min 8s


In [66]:
df_temp = df[['flair_sentiment_tag', 'flair_sentiment_score', 'flair_preprocessed_sentiment_tag', 'flair_preprocessed_sentiment_score']]

In [79]:
# Get statistics for flair method WITHOUT preprocessing
print(df_temp['flair_sentiment_tag'].value_counts())

df_temp['flair_sentiment_score'].describe().to_frame()

NEGATIVE    2133
POSITIVE     511
Name: flair_sentiment_tag, dtype: int64


,flair_sentiment_score
count,2644.000000
mean,0.863480
std,0.141524
min,0.500408
25%,0.782573
50%,0.920497
75%,0.979667
max,0.999985


In [80]:
# Get statistics for flair method WITH preprocessing
print(df_temp['flair_preprocessed_sentiment_tag'].value_counts())

df_temp['flair_preprocessed_sentiment_score'].describe().to_frame()

POSITIVE    2419
NEGATIVE     225
Name: flair_preprocessed_sentiment_tag, dtype: int64


,flair_preprocessed_sentiment_score
count,2644.000000
mean,0.888619
std,0.124030
min,0.501601
25%,0.845084
50%,0.939931
75%,0.979666
max,0.999345


In [62]:
s = flair.data.Sentence("The dog died")
flair_sentiment.predict(s)
s.tag, s.score

('NEGATIVE', 0.9997033476829529)